In [55]:
import os
import numpy as np
import cv2
from glob import glob
from multiprocessing import Pool
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [56]:
_IMAGE_SIZE = 256

In [57]:
def cal_for_frames(video_path):
    print(video_path)
    frames = glob(os.path.join(video_path, '*.jpg'))
    frames.sort()

    flow = []
    prev = cv2.imread(frames[0])
    prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    prev = cv2.resize(prev,(224,224))
    for i, frame_curr in enumerate(frames):
        curr = cv2.imread(frame_curr)
        curr = cv2.resize(curr,(224,224))
        curr = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
        tmp_flow = compute_TVL1(prev, curr)
        flow.append(tmp_flow)
        prev = curr

    return flow


In [58]:

def compute_TVL1(prev, curr, bound=15):
    """Compute the TV-L1 optical flow."""
    TVL1 = cv2.optflow.DualTVL1OpticalFlow_create()
    flow = TVL1.calc(prev, curr, None)
    flow = np.clip(flow, -20,20) #default values are +20 and -20
    #print(flow)
    assert flow.dtype == np.float32

    flow = (flow + bound) * (255.0 / (2*bound))
    flow = np.round(flow).astype(int)
    flow[flow >= 255] = 255
    flow[flow <= 0] = 0

    return flow

In [59]:
def save_flow(video_flows, flow_path):
    #The optical flows are generated in 3D. However for I3D only first two channels are used. u is the first channel 
    #and v is the second channel. Both u and v are saved in separate folders in the flow_path directory. 
    #The u and v folders will be generated by calling create_path() function
    n_py=[]
    for i, flow in enumerate(video_flows):
        cv2.imwrite(os.path.join(flow_path.format('u'), "{:06d}.jpg".format(i)),
                    flow[:, :, 0])
        cv2.imwrite(os.path.join(flow_path.format('v'), "{:06d}.jpg".format(i)),
                    flow[:, :, 1])
        n_py.append(np.dstack([flow[:, :, 0],flow[:, :, 1]]))
    return n_py


In [60]:
def extract_flow(args):
    video_path, flow_path = args
    flow = cal_for_frames(video_path)
    save_flow(flow, flow_path)
    print('complete:' + flow_path)
    return

In [61]:
def create_paths(base_path,activity):
    activity_path=os.path.join(base_path,activity)
    rgb_folder=os.makedirs('{}/{}_rgb'.format(activity_path,activity))
    flow_folder1=os.makedirs('{}/{}_flow/u'.format(activity_path,activity))
    flow_folder2=os.makedirs('{}/{}_flow/v'.format(activity_path,activity))
    

In [62]:
def vid_to_image(base_path,file,frame_path):
    folder=file.split('.')[0]
    #print(folder)
    #os.mkdir(os.path.join(base_path,folder))
    vidcap = cv2.VideoCapture(file)
    success,image = vidcap.read()
    count = 0
    frame_no=0
    print(frame_path)
    while success:
            vidcap.set(1,frame_no)
            #print(os.path.join(frame_path,"frame%d.jpg"))
            cv2.imwrite(os.path.join(frame_path,"frame%d.jpg") % count, image)     # save frame as JPEG file     
            success,image = vidcap.read()
            #print('Read a new frame: ', success)
            count += 1
            frame_no+=1

In [63]:
#name of activity what is in the video. This is just for purpose of creating proper folders.

activity='cricket'

#This is the path where your video is kept. for simplicity rename the video same as activity. for eg. laughing.mp4
base_path='/home/lenovo/object-detection/spot-on/I3D/kinetics-i3d/convert_vid2npy/train/'

In [64]:
#this function creates folders u and v which store the optical flow images
create_paths(base_path,activity)

#This is the path where  images sampled from the video are to be saved
frame_path='/home/lenovo/object-detection/spot-on/I3D/kinetics-i3d/convert_vid2npy/train/{}/{}_rgb/'.format(activity,activity) 

# this is the path where you want save the flow files.
flow_path='/home/lenovo/object-detection/spot-on/I3D/kinetics-i3d/convert_vid2npy/train/{}/{}_flow/{}'.format(activity,activity,{}) 

In [ ]:
#Convert your video to frames and save in to activity_rgb folder
vid_to_image(base_path,os.path.join(base_path,activity+'.mp4'),frame_path)


/home/lenovo/object-detection/spot-on/I3D/kinetics-i3d/convert_vid2npy/train/cricket/cricket_rgb/


In [28]:
#calculate flows
flow = cal_for_frames(frame_path)

/home/lenovo/object-detection/spot-on/I3D/kinetics-i3d/convert_vid2npy/train/cricket_samp/cricket_samp_rgb/


In [46]:
#save flows to folders u and v
npy_flow=save_flow(flow, flow_path)

# Convert images to .npyfile as descibed in  https://github.com/deepmind/kinetics-i3d

In [41]:
# scaler = MinMaxScaler(feature_range=(-1,1))
# print(scaler.transform(data))

In [50]:
## Create npy file for rgb files as described in deepmind I3D
def norm_rgb(rgb_path,nchannel):
    #print(video_path)
    npy_file=[]
    frames = glob(os.path.join(rgb_path, '*.jpg'))
    frames.sort()
    print(len(frames))
    for frame in frames:
        img=cv2.imread(os.path.join(frame_path,frame))
        img_new=(cv2.resize(img,(224,224))).astype(float)
        img_norm=np.divide(2*(img_new-img_new.min()),(img_new.max()-img_new.min()))-1
        
        npy_file.append(img_norm)
        
    npy_file=np.reshape(np.asarray(npy_file),(1,len(frames),224,224,nchannel))
    np.save('data_input_rgb_{}.npy'.format(activity),npy_file)
    return npy_file
        

In [51]:
## Create npy file of flow files as described in deepmind I3D
def norm_flow(rgb_path,nchannel):
    #print(video_path)
#     npy_file=[]
    frames = glob(os.path.join(rgb_path, '*.jpg'))
    frames.sort()
    print(len(frames))
#     for frame in frames:
#         img=cv2.imread(os.path.join(frame_path,frame),0)
#         #img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         img_new=(cv2.resize(img,(224,224))).astype(float)
#         img_norm=np.divide(2*(img_new-img_new.min()),(img_new.max()-img_new.min()))-1
        
#         npy_file.append(img_norm[:,:,:-1])
        
    npy_file=np.reshape(np.asarray(npy_flow),(1,len(frames),224,224,nchannel-1)).astype(float)
    #clip between range [0,40]
    #npy_file=np.clip(npy_file,-20,20)
    #rescale betwwen [-1,1]
    npy_file=((2*(npy_file-npy_file.min())/(npy_file.max()-npy_file.min()))-1)
    np.save('data_input_flow_{}.npy'.format(activity),npy_file)
    return npy_file

In [53]:
np_file=norm_rgb(frame_path,3)

79


In [54]:
np_file=norm_flow(frame_path,3)

79
